# Assignment 2 Cox Regression Analysis with Time-Varying Covariates 

*Due February 28*


In this assignment you will learn to estimate the risks associated with covariates that change over time, such as RNA measurements. The basics of Cox regression are in Chapter 3 of “Applied Survival Analysis”. 

## Conceptual Questions

**1)** What are the advantages and disadvantages of Kaplan-Meier vs Cox Regression Analysis?

| |Advantages|Disadvantages|
|--|--|--|
|Kaplan-Meier|Simple model to implement.|Cannot use multiple parameters. Cannot handle hazard.|
|Cox Regression|Can use multiple parameters. Can handle hazard.| |



**2)** What additional implementation issues arise when implementing time-varying covariates in Cox regression vs. baseline covariates only?

*reference Time-varying covariates in the documentation to the python package “lifelines” https://lifelines.readthedocs.io/en/latest/Time%20varying%20survival%20regression.html*

## Coding part:

Here we will implement a function for performing Cox regression, and then extend it to time-varying covariates.

### Step 1) Test data simulation

**1a)** Pick a set of test beta-coefficients (= log Hazard Ratios)
for a set of 2-3 “predictors” with data matrix X. You can generate the entries of X with a standard normal distribution.


In [13]:
using DataFrames

df = DataFrame(
    Event = [0, 1, 1, 1],
    Time = [2, 4, 4, 8], 
    betaX = 1:4
)

,Event,Time,betaX
,Int64,Int64,Int64
1,0,2,1
2,1,4,2
3,1,4,3
4,1,8,4


**1b)** Create a random failure CDF according to the method of Harden and Kropko (see Generating the Baseline Hazard Function). Plot your baseline survival function (1 - failure CDF).

In [14]:
using Distributions
using PlotlyJS

# create a uniform vector between 1 and 10
T_max = 100
k = 10
c = zeros(1)

append!( c, sort(rand(Uniform(0, 9), 8)), k)

xs = 1:k
ys = c

plot(scatter(x=xs, y=ys, mode="markers"),
     Layout(yaxis_title="Cumulative P(Failure)", xaxis_title="Time", title="Random unifor distrubuted points"))

data: [
  "scatter with fields mode, type, x, and y"
]

layout: "layout with fields margin, template, title, xaxis, and yaxis"

In [15]:
# Construct failure CDF by fitting a cubic smoothing spline - Hymans cubic smoothing function
using PCHIPInterpolation

println(length(xs))
println(length(ys))

itp = Interpolator(xs, ys)

x_range = range(xs[1], stop=xs[end], length=100)
y_interpolated = [itp(x) for x in x_range]

trace_org = scatter(x=xs, y=ys, mode="markers")

trace_itp = scatter(x=x_range, y=y_interpolated, mode="lines")

plot([trace_org, trace_itp], Layout(yaxis_title="Cumulative P(Failure)", xaxis_title="Time", title="Cubic spline (failure CDF)"))


10
10


data: [
  "scatter with fields mode, type, x, and y",
  "scatter with fields mode, type, x, and y"
]

layout: "layout with fields margin, template, title, xaxis, and yaxis"

**1c)** For each subject in your study, generate a synthetic survival time (see Harden and Kropko Generating Durations). We can assume for now that there is no censoring. Note that a survival function is a CDF for the event-time. This means that you can use simple inverse sampling to get survival time samples. 

In [31]:
# get PDF of failure CDF by using the gradient for the failure CDF
using Interpolations
using ShiftedArrays
#using TimeSeries
#using Pandas

df = DataFrame(failure_cdf=y_interpolated)
df = transform(df, :failure_cdf => (x -> x - lag(x)) => :failure_pdf)
df = transform(df, :failure_cdf => (x -> (1 .- x)) => :survivor)
println(df)



100×3 DataFrame
 Row │ failure_cdf  failure_pdf      survivor   
     │ Float64      Float64?         Float64    
─────┼──────────────────────────────────────────
   1 │    0.0       missing           1.0
   2 │    0.154038        0.154038    0.845962
   3 │    0.30375         0.149712    0.69625
   4 │    0.447837        0.144087    0.552163
   5 │    0.585           0.137163    0.415
   6 │    0.713942        0.128941    0.286058
   7 │    0.833363        0.119421    0.166637
   8 │    0.941966        0.108603    0.0580342
   9 │    1.03845         0.0964856  -0.0384514
  10 │    1.12152         0.0830702  -0.121522
  11 │    1.18988         0.0683564  -0.189878
  12 │    1.24222         0.0523442  -0.242222
  13 │    1.28177         0.039551   -0.281773
  14 │    1.31375         0.0319786  -0.313752
  15 │    1.33986         0.0261102  -0.339862
  16 │    1.36181         0.0219459  -0.361808
  17 │    1.38129         0.0194856  -0.381294
  18 │    1.40002         0.0187294  -0.40002

### Step 2) Cox model implementation

**2a)** Create a test function which calls your to be implemented Cox model with your test survival data. You should assert that the estimated hazard ratio values are reasonably close to the ones you used to create your test data.

**2b)** Implement a Cox regression model that estimates hazard ratios based on event-time data (duration, censoring). You can use a standard optimization package to perform likelihood maximisation. A Newton-type method such as L-BFGS-B or SQP should be sufficient. The likelihood function should hopefully be calculated numerically by the optimization routine. Otherwise you can get it by finite-differencing or algorithmic differentiation. 

**2c)** Test your new Cox Regression model function with your test function.

### Step 3) Extension

**3a)** Add a random censoring mechanism to your test data. You will need to pick a distribution function for censoring that depends on time. Example functions are uniform (simple) or exponential (to cluster the failure time at the beginning or end of the interval).  An easy way to control the amount of censoring is to extend the censoring PDF to times past the limit of your survival PDF. Check that your Cox model still estimates reasonable hazard ratios given a large enough synthetic sample size.

**3b)** Create a new test data function and cox regression model for time-varying covariates. You can copy and update your previous functions.
You will need to allow your data X to vary over time. This can be achieved with a stochastic process like the Ornstein-Uhlenbeck model, which allows us to control the amount of time-variation in each covariate.

https://ipython-books.github.io/134-simulating-a-stochastic-differential-equation/

To get the subject specific survival function you can numerically integrate the baseline hazard function multiplied by (X(t) dot beta) over time to get a hazard function. The survival function is then exp(-hazard).
